We assume there are only two predictors $X_0$ and $X_1$. Points in class $0$ fall on the line segment $X_1 = X_0 + 20$ with $X_0 \in [0,100]$. Points in class $1$ fall on the line segment $X_1 = X_0 - 20$ with $X_0 \in [20,120]$. Please note that in both cases $X_0$ are uniformly distributed. We assume $X_0$ and $X_1$ are comonotonic. In this notebook, we compare CIBer with Naive Bayes.
Let's first think of an example. If we would like to classify point $(55,35)$. We know that it should be class $1$ since it falls on the line segment $X_1 = X_0 - 20$. Furtherly, suppose we discretize $X_0$ into equal bins with length $10$. Then each conditional marginal probability equals to $0.1$.

$\textbf{Naive Bayes}$

$\mathbb{P}(X_0,X_1|Y=0)\cdot \mathbb{P}(Y=0)=0.1\cdot0.1\cdot0.5=0.0005$

$\mathbb{P}(X_0,X_1|Y=1)\cdot \mathbb{P}(Y=1)=0.1\cdot0.1\cdot0.5=0.0005$

$\textbf{CIBer}$

$\mathbb{P}(X_0,X_1|Y=0)\cdot \mathbb{P}(Y=0)= Leb([0.5,0.6]\cap[0.1,0.2])\cdot0.5=0$

$\mathbb{P}(X_0,X_1|Y=1)\cdot \mathbb{P}(Y=1)= Leb([0.3,0.4]\cap[0.3,0.4])\cdot0.5=0.05$

So we can see that Naive Bayes can not make a decision but CIBer will definitely predict it as class $1$.

In [1]:
import numpy as np
import pandas as pd
import comonotonic as cm
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import utils
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, roc_auc_score
from multiprocessing import Pool
import time
import xlsxwriter
import copy
import math
import statistics

In [2]:
# here n_features stands for the pairs of features!!!
def experiment(bin_num, n_features):
    n_sample_each_class = 5000
    data0 = list()
    data1 = list()
    for i in range(n_features):
        underlying0 = np.random.uniform(0,100,n_sample_each_class)
        derivative0 = underlying0 + 20
        data0.append(underlying0)
        data0.append(derivative0)

        underlying1 = np.random.uniform(20,120,n_sample_each_class)
        derivative1 = underlying1 - 20
        data1.append(underlying1)
        data1.append(derivative1)
    data0 = np.array(data0).T
    data1 = np.array(data1).T
    
    class0 = np.array([0 for i in range(n_sample_each_class)]).reshape(-1,1)
    class1 = np.array([1 for i in range(n_sample_each_class)]).reshape(-1,1)
    class0 = np.concatenate((data0, class0), axis = 1)
    class1 = np.concatenate((data1, class1), axis = 1)
    data = np.concatenate((class0, class1), axis = 0)
    X = data[:,:-1]
    Y = data[:,-1]
    cont_col = [i for i in range(X.shape[1])]
    categorical = []
    discrete_feature_val = None
    allocation_book = {i:bin_num for i in cont_col}
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,shuffle = True)
    
    time1 = time.time()
    ciber = cm.clustered_comonotonic(X_train,Y_train,discrete_feature_val,cont_col,categorical,
                                     0.6,None,corrtype = 'mutual_info',discrete_method = "custom",
                                     allocation_book = allocation_book)
    ciber.run()
    time2 = time.time()
    discretized_x_test = ciber.discretize_test(X_test)
    time3 = time.time()
    ciber_predict = ciber.predict(discretized_x_test)
    time4 = time.time()
    ciber_train = time2 - time1
    ciber_test = time4 - time3
    
    time5 = time.time()
    ciber_nb = cm.clustered_comonotonic(X_train,Y_train,discrete_feature_val,cont_col,categorical,
                                        1,None,corrtype = 'mutual_info',discrete_method = "custom",
                                        allocation_book = allocation_book)
    ciber_nb.run()
    time6 = time.time()
    
    time7 = time.time()
    ciber_nb_predict = ciber_nb.predict(discretized_x_test)
    time8 = time.time()
    nb_train = time6 - time5
    nb_test = time8 - time7
    
    return ciber_train, ciber_test, nb_train, nb_test

In [3]:
def repeat_experiment(bin_num, n_features, n_repeat):
    ciber_train_time = []
    ciber_test_time = []
    nb_train_time = []
    nb_test_time = []
    params = [(bin_num,n_features) for i in range(n_repeat)]
    pool = Pool()
    results = pool.starmap(experiment, params)
    pool.close()
    for result in results:
        ciber_train_time.append(result[0])
        ciber_test_time.append(result[1])
        nb_train_time.append(result[2])
        nb_test_time.append(result[3])
    return ciber_train_time, ciber_test_time, nb_train_time, nb_test_time

In [4]:
bin_num = 10
features = [i+2 for i in range(10)]
n_repeat = 10
c_train = dict()
c_test = dict()
n_train = dict()
n_test = dict()
for n_features in features:
    a,b,c,d = repeat_experiment(bin_num, n_features, n_repeat)
    c_train[n_features*2] = a.copy()
    c_test[n_features*2] = b.copy()
    n_train[n_features*2] = c.copy()
    n_test[n_features*2] = d.copy()
    del a,b,c,d

Process ForkPoolWorker-92:
Process ForkPoolWorker-127:
Process ForkPoolWorker-78:
Process ForkPoolWorker-88:
Process ForkPoolWorker-126:
Process ForkPoolWorker-99:
Process ForkPoolWorker-101:
Process ForkPoolWorker-125:
Process ForkPoolWorker-96:
Process ForkPoolWorker-123:
Process ForkPoolWorker-122:
Process ForkPoolWorker-124:
Process ForkPoolWorker-82:
Process ForkPoolWorker-120:
Process ForkPoolWorker-69:
Process ForkPoolWorker-119:
Traceback (most recent call last):
Process ForkPoolWorker-94:
Process ForkPoolWorker-79:
Process ForkPoolWorker-86:
Traceback (most recent call last):
Process ForkPoolWorker-118:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-117:
Process ForkPoolWorker-91:
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-116:
Process ForkPoolWorker-68:
Process ForkPoolWorker-121:
Process ForkPoolWorker-

Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-74:
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most r

  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/proce

  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
KeyboardInterrupt
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File

  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in

  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/home/r7user5/Desktop/STAT/simulation/comonotonic.py", line 325, in run
    self.clustering()
KeyboardInterrupt
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/home/r7user5/Desktop/STAT/simulation/comonotonic.py", line 325, in run
    self.clustering()
KeyboardInterrupt
  File "/home/r7user5/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrup

KeyboardInterrupt: 

In [ ]:
def compute_CI(ll):
    mean = statistics.mean(ll)
    std = statistics.stdev(ll)
    t_value = 2.2622
    lower = mean - t_value * std / math.sqrt(9)
    upper = mean + t_value * std / math.sqrt(9)
    return mean, lower, upper

In [ ]:
workbook1 = xlsxwriter.Workbook('c_train1.xlsx')
workbook2 = xlsxwriter.Workbook('c_test1.xlsx')
workbook3 = xlsxwriter.Workbook('n_train1.xlsx')
workbook4 = xlsxwriter.Workbook('n_test1.xlsx')
worksheet1 = workbook1.add_worksheet()
worksheet2 = workbook2.add_worksheet()
worksheet3 = workbook3.add_worksheet()
worksheet4 = workbook4.add_worksheet()

title = ['n_feature','mean','T lower','T upper'] + ["time "+str(i+1) for i in range(n_repeat)]
for i,t in enumerate(title):
    worksheet1.write(0, i, t)
    worksheet2.write(0, i, t)
    worksheet3.write(0, i, t)
    worksheet4.write(0, i, t)

row = 1
for f in c_train.keys():
    worksheet1.write(row, 0, f)
    m1, l1, u1 = compute_CI(c_train[f])
    worksheet1.write(row, 1, m1)
    worksheet1.write(row, 2, l1)
    worksheet1.write(row, 3, u1)
    for i, t in enumerate(c_train[f]):
        worksheet1.write(row, i+4, t)
    
    worksheet2.write(row, 0, f)
    m2, l2, u2 = compute_CI(c_test[f])
    worksheet2.write(row, 1, m2)
    worksheet2.write(row, 2, l2)
    worksheet2.write(row, 3, u2)
    for i, t in enumerate(c_test[f]):
        worksheet2.write(row, i+4, t)

    worksheet3.write(row, 0, f)
    m3, l3, u3 = compute_CI(n_train[f])
    worksheet3.write(row, 1, m3)
    worksheet3.write(row, 2, l3)
    worksheet3.write(row, 3, u3)
    for i, t in enumerate(n_train[f]):
        worksheet3.write(row, i+4, t)

    worksheet4.write(row, 0, f)
    m4, l4, u4 = compute_CI(n_test[f])
    worksheet4.write(row, 1, m4)
    worksheet4.write(row, 2, l4)
    worksheet4.write(row, 3, u4)
    for i, t in enumerate(n_test[f]):
        worksheet4.write(row, i+4, t)
    
    row += 1

workbook1.close()
workbook2.close()
workbook3.close()
workbook4.close()